### Import relevant libraries

In [1]:
import os
import pandas as pd
import numpy as np

from nilearn import image
from nilearn import input_data
from nilearn import signal
from nilearn import connectome

from bids import BIDSLayout
from load_confounds import Params9
from load_confounds import AnatCompCor

import process_bids as pb
import process_connectome as pc

pd.options.mode.chained_assignment = None

### Useful Functions

### Load input data

In [5]:
dataset_name = 'msc'
raw_path = os.path.abspath('D:\msc-raw')
derivatives_path = os.path.abspath(r'D:\msc-derivatives\fmriprep')

# Lecture du format BIDS
layout = BIDSLayout(raw_path, derivatives = derivatives_path, validate = True,
                    database_path='pybids_True.sql', index_metadata=True, reset_database=False)

layout

BIDS Layout: ...D:\msc-raw | Subjects: 0 | Sessions: 0 | Runs: 0

### Input variables

In [6]:
subjects_id_list = layout.get_subjects()
t_r = layout.get_tr()
print('Subjects ID:', subjects_id_list)

Subjects ID: ['MSC01', 'MSC02', 'MSC03', 'MSC04', 'MSC05', 'MSC06', 'MSC07', 'MSC08', 'MSC09', 'MSC10']


### Load brain parcellation

In [ ]:
atlas_name = 'CAB-NP'
atlas_path = os.path.abspath(r'..\..\atlas\CAB-NP_volumetric\CAB-NP_volumetric_liberal.nii.gz')
atlas_masker = input_data.NiftiLabelsMasker(atlas_path, standardize=False, smoothing_fwhm=6, low_pass=0.01, t_r=t_r)

### Parallele processing for time series calculation

In [7]:
time_series_info = pd.DataFrame(columns = subjects_id_list, index = ['path','nlevels','atlas'])
time_series_info.loc['atlas',:] = atlas_name

In [8]:
directory = 'timeSeries_files'

sub_dir = os.path.join(directory,dataset_name,atlas_name)
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

confounds_name = ['None', 'params9','aCompCor']

In [ ]:
for id in subjects_id_list:
    csv_path = os.path.join(sub_dir ,'id_atlas_timeSeries.csv'.replace('id',id).replace('atlas',atlas_name))

    sessions_sub = pb.get_sessions(layout, id)
    tasks_by_session_sub = pb.get_tasks(layout, id, sessions_sub)
    run_by_task_sub = pb.get_run(layout, id, sessions_sub, tasks_by_session_sub)
    subj_timeseries_df = pc.create_empty_df_timesseries(sessions_sub,tasks_by_session_sub, run_by_task_sub, confounds_name)

    # Calcul sur l'ensemble des nifti
    subjects_im_path = layout.get(subject = id, extension='nii.gz', suffix='bold', scope='derivatives',
                                  return_type='filename')

    params9 = Params9().load(subjects_im_path)
    aCompCor = AnatCompCor().load(subjects_im_path)


    time_series_info.loc['path',id] = csv_path
    time_series_info.loc['nlevels',id] = subj_timeseries_df.columns.nlevels
    subj_timeseries_df = pc.calculate_timeseries(atlas_masker, run_by_task_sub, subjects_im_path, csv_path,
                                                subj_timeseries_df, t_r, subject_confounds_list = [params9,aCompCor],
                                                confounds_name= confounds_name)

In [ ]:
info_csv_path = os.path.join(sub_dir,'timeSeries_info.csv')
time_series_info.to_csv(info_csv_path, header= True)
time_series_info

### Subjects caracteristiques

In [ ]:
time_series_info = pd.read_csv(info_csv_path, header = 0, index_col=0)
time_series_info

### Connectomes calculation

In [ ]:
directory = 'connectomes'

sub_dir = os.path.join(directory,dataset_name, atlas_name)
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

confounds_name.remove('None')

for confound in confounds_name:
    conf_dir = os.path.join(sub_dir,confound)
    if not os.path.exists(conf_dir):
        os.makedirs(conf_dir)

In [ ]:
kind_connect = 'correlation'
vectorize_connect = True
discard_diagonal_connect = True

pc.extract_connectomes(time_series_info, sub_dir, confounds_name, kind=kind_connect, vectorize=vectorize_connect,
                       discard_diagonal=discard_diagonal_connect)
